In [1]:
import copy
import os
import argparse
import torch
import numpy as np
import random

%load_ext autoreload
%autoreload 2
from erasure.utils.config.local_ctx import Local
from erasure.utils.config.global_ctx import Global 
from erasure.core.factory_base import ConfigurableFactory
from erasure.data.datasets.DatasetManager import DatasetManager

/NFSHOME/adangelo/miniconda3/envs/representer/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_file = os.path.join("configs", "gnn", "Reddit-threads.jsonc")
#config_file = os.path.join("configs", "example.jsonc")

global_ctx = Global(config_file)
global_ctx.factory = ConfigurableFactory(global_ctx)

#Create Dataset
data_manager = global_ctx.factory.get_object( Local( global_ctx.config.data ))
global_ctx.dataset = data_manager

2025-02-17 15:03:29,339254830 | INFO | 664461 - Creating Global Context for: configs/gnn/Reddit-threads.jsonc
2025-02-17 15:03:29,339254953 | INFO | 664461 - Setting seeds to: 0


2025-02-17 15:03:29,339255112 | INFO | 664461 - Caching System: True.
2025-02-17 15:03:30,339255237 | INFO | 664461 - Instantiating: torch_geometric.datasets.TUDataset


Extracting resources/data/MUTAG/MUTAG.zip


2025-02-17 15:03:30,339255802 | INFO | 664461 - Created Configurable: erasure.data.data_sources.TorchGeometricDataSource.TorchGeometricDataSource
2025-02-17 15:03:30,339255803 | INFO | 664461 - {'class': 'erasure.data.data_sources.TorchGeometricDataSource.TorchGeometricDataSource', 'parameters': {'datasource': {'class': 'torch_geometric.datasets.TUDataset', 'parameters': {'root': 'resources/data', 'name': 'MUTAG'}, 'preprocess': []}}}
MUTAG(188)
Data(edge_index=[2, 38], x=[17, 7], edge_attr=[38, 4], y=[1])
2025-02-17 15:03:30,339255852 | INFO | 664461 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage


Processing...
Done!


2025-02-17 15:03:30,339255899 | INFO | 664461 - ['all', 'all_shuffled', '-']
2025-02-17 15:03:30,339255900 | INFO | 664461 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage
2025-02-17 15:03:30,339255901 | INFO | 664461 - ['all', 'all_shuffled', '-', 'train', 'test']
2025-02-17 15:03:30,339255901 | INFO | 664461 - Instantiating: erasure.data.datasets.DataSplitter.DataSplitterPercentage
2025-02-17 15:03:30,339255902 | INFO | 664461 - ['all', 'all_shuffled', '-', 'train', 'test', 'forget', 'retain']
2025-02-17 15:03:30,339255902 | INFO | 664461 - ['all', 'all_shuffled', '-', 'train', 'test', 'forget', 'retain']
2025-02-17 15:03:30,339255903 | INFO | 664461 - Created Configurable: erasure.data.datasets.DatasetManager.DatasetManager


In [3]:
data_manager.n_classes

2

In [13]:
from erasure.utils.config.global_ctx import bcolors

#Create Predictor
current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.predictor = predictor
global_ctx.logger.info('Global Predictor: ' + str(predictor))

2025-02-17 11:08:07,325132501 | INFO | 569228 - Instantiating: erasure.model.gcn.DownstreamGCN
2025-02-17 11:08:07,325132599 | INFO | 569228 - Instantiating: torch.optim.RMSprop
2025-02-17 11:08:07,325132600 | INFO | 569228 - Instantiating: torch.nn.CrossEntropyLoss
2025-02-17 11:08:07,325132602 | INFO | 569228 - TRAINING WITH 1210 samples
2025-02-17 11:08:07,325132833 | INFO | 569228 - epoch = 0 ---> loss = 0.7260	 accuracy = 0.5356
2025-02-17 11:08:07,325133062 | INFO | 569228 - epoch = 1 ---> loss = 0.6887	 accuracy = 0.5425
2025-02-17 11:08:08,325133307 | INFO | 569228 - epoch = 2 ---> loss = 0.6871	 accuracy = 0.5451
2025-02-17 11:08:08,325133547 | INFO | 569228 - epoch = 3 ---> loss = 0.6861	 accuracy = 0.5451
2025-02-17 11:08:08,325133774 | INFO | 569228 - epoch = 4 ---> loss = 0.6853	 accuracy = 0.5469
2025-02-17 11:08:08,325134005 | INFO | 569228 - epoch = 5 ---> loss = 0.6846	 accuracy = 0.5460
2025-02-17 11:08:09,325134303 | INFO | 569228 - epoch = 6 ---> loss = 0.6839	 accu

KeyboardInterrupt: 

In [ ]:

#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )


#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating: {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

In [ ]:
forget_loader, _ = data_manager.get_loader_for('forget')
len(forget_loader)

66

In [18]:
train_loader, _ = data_manager.get_loader_for('train')

2025-02-17 11:17:48,325713638 | INFO | 569228 - TRAINING WITH 1210 samples


In [19]:
forget = data_manager.partitions['forget']
len(forget)

242

In [20]:
for X,y in train_loader:
    print(X)
    print(y)

DataBatch(x=[165, 9], edge_index=[2, 356], edge_attr=[356, 3], batch=[165], ptr=[5])
tensor([1, 1, 1, 0])
DataBatch(x=[157, 9], edge_index=[2, 336], edge_attr=[336, 3], batch=[157], ptr=[5])
tensor([1, 1, 0, 1])
DataBatch(x=[107, 9], edge_index=[2, 234], edge_attr=[234, 3], batch=[107], ptr=[5])
tensor([0, 0, 0, 0])
DataBatch(x=[124, 9], edge_index=[2, 272], edge_attr=[272, 3], batch=[124], ptr=[5])
tensor([1, 0, 0, 1])
DataBatch(x=[136, 9], edge_index=[2, 290], edge_attr=[290, 3], batch=[136], ptr=[5])
tensor([1, 1, 0, 1])
DataBatch(x=[133, 9], edge_index=[2, 286], edge_attr=[286, 3], batch=[133], ptr=[5])
tensor([1, 1, 0, 0])
DataBatch(x=[170, 9], edge_index=[2, 366], edge_attr=[366, 3], batch=[170], ptr=[5])
tensor([0, 1, 1, 1])
DataBatch(x=[123, 9], edge_index=[2, 268], edge_attr=[268, 3], batch=[123], ptr=[5])
tensor([0, 0, 1, 1])
DataBatch(x=[117, 9], edge_index=[2, 254], edge_attr=[254, 3], batch=[117], ptr=[5])
tensor([0, 0, 0, 1])
DataBatch(x=[135, 9], edge_index=[2, 294], edg

In [ ]:
from transformers import BertTokenizer

# Load the same tokenizer used for tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Iterate through your DataLoader
for X, y in forget_loader:
    decoded_text = tokenizer.batch_decode(X[0], skip_special_tokens=True)
    
    print(decoded_text)


['with easy operation you can make sweets happily you can learn how to make sweets with over 100 kinds of fun while playing there are many recipes including cakes cookies japanese sweets and so on', '[unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0]']
['there is no one in the world who could return from the dark side inviolate darkness and flame the dark side is an adventure game in the genre of hidden objects with plenty of minigames and puzzles unforgettable characters and complicated quests', '[unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0] [unused0

In [ ]:
import matplotlib.pyplot as plt

# Parameters
num_images_to_display = 40
images_per_row = 4  # Number of images per row for a grid layout
images_displayed = 0

# Collect and display images
plt.figure(figsize=(15, 10))  # Set the figure size

for X, y in forget_loader:
    # Convert the batch to NumPy
    np_images = X.permute(0, 2, 3, 1).numpy()  # Change shape to (N, H, W, C)
    
    # Loop through images in the batch
    for img, label in zip(np_images, y):
        images_displayed += 1
        
        # Add subplot for each image
        plt.subplot(num_images_to_display // images_per_row, images_per_row, images_displayed)
        plt.imshow(img)  # Assuming the range is already suitable for visualization
        plt.title(f"Label: {label.item()}")  # Display label
        plt.axis("off")
        
        if images_displayed == num_images_to_display:
            break  # Stop after displaying the required number of images
    if images_displayed == num_images_to_display:
        break

plt.tight_layout()  # Adjust spacing between subplots
plt.savefig('2820_new_new.png')
plt.show()


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 3 is not equal to len(dims) = 4

<Figure size 1500x1000 with 0 Axes>

In [ ]:
loader, _ = data_manager.get_loader_for('train')

2025-01-31 17:09:43,-1121971173 | INFO | 346018 - TRAINING WITH 124594 samples


In [ ]:

from erasure.utils.config.global_ctx import bcolors


#Create Predictor
current = Local(global_ctx.config.predictor)
current.dataset = data_manager
predictor = global_ctx.factory.get_object(current)
global_ctx.predictor = predictor
global_ctx.logger.info('Global Predictor: ' + str(predictor))

#Create unlearners 
unlearners = []
unlearners_cfg = global_ctx.config.unlearners
for un in unlearners_cfg:
    current = Local(un)
    current.dataset = data_manager
    current.predictor = copy.deepcopy(predictor)
    unlearners.append( global_ctx.factory.get_object(current) )


#Evaluator
current = Local(global_ctx.config.evaluator)
current.unlearners = unlearners
evaluator = global_ctx.factory.get_object(current)

# Evaluations
for unlearner in unlearners:
    global_ctx.logger.info(f'''{bcolors.OKGREEN}####\t\t Evaluating: {unlearner.__class__.__name__} \t\t####{bcolors.ENDC}''')
    evaluator.evaluate(unlearner,predictor)

2025-02-14 15:07:59,80324598 | INFO | 3335432 - Instantiating: erasure.model.gcn.DownstreamGCN


TypeError: __init__() missing 1 required positional argument: 'node_features'